# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [20]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [21]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [22]:
browser = webdriver.Chrome('../chromedriver.exe')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load the page.
    time.sleep(2)
    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
    print(new_height)

    if new_height == last_height:
        break
    last_height = new_height

11330
13645
15983
18269
20533
22855
24198
24248
24198
24198


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [23]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'html.parser')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [24]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.text))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 晶呈科技5月合併營收5,560萬元　年增率21.77％

▲晶呈科技5月合併營收5,560萬元，圖為發言人張永宏。（圖／資料照）
記者高兆麟／綜合報導
半導體級特殊氣體製造大廠晶呈科技(4768)，今日公告2021年5月營收5,560萬元，月增5.8％、年增21.77％雙成長，累計前5月營收2億4,603萬元，年增率27.93％。係新冠肺炎疫情期間，5G、物聯網、居家辦公/學習需求強勁，帶動消費性電子的需求商機，受惠半導體產能供不應求下，推升公司特殊氣體的需求成長所致。
晶呈5月營收5,560萬元，月增率5.8％，公司表示，雖台灣本土疫情五月急速升溫，但公司在完善的防疫措施下，生產運作正常；隨著半導體往先進製程演進，特殊氣體的需求提升下，公司除打入台灣各大重要半導體大廠外，亦外銷至美、日、韓、中等半導體生產國，成功將公司從貿易代理轉型為專業之半導體先進材料製造商，並大幅提升自製商品占比從2018年54％攀升至2020年75％，受惠疫情帶動遠距商機需求強勁，半導體產能供不應求下，推升5月營收年增率21.77％。
晶呈製造之特殊氣體，主要應用於半導體與平面顯示器等製程所需，憑藉在製程、容器製作、純化與微量分析之關鍵技術，產品品質掌控度高，在地供應鏈、快速提供給客戶等優勢，爭取客戶訂單量增加，未來依據客戶需求成長興建廠房增加產能，以期滿足客戶成長需求。
因應半導體蓬勃發展，晶圓廠擴廠增量，為提高矽晶圓的使用壽命，晶呈提供之晶圓重生服務讓準報廢矽晶圓再度使用，藉此達到效益極大化，相較於傳統晶圓再生以液態除膜方案，晶呈自研成功的晶圓重生技術，去膜成本低、重生能力與拯救率高，不僅是矽晶圓表面除膜最佳的環保節能解決方案，挾技術領先優勢，製程優化，回收率提升下，以期成為公司新的營運成長引擎。
此外，晶呈開發之特殊金屬複合材料銅磁晶片，具備順磁性、高散熱、低熱膨脹係數、超薄之特性，透過上下游設備與製程整合，開發出相關Min & Micro LED應用產品，非常適用於在 LED chip 微型化的發展，可應用在照明、燈號、車用、透明與不透明顯示屏，藉由銅磁晶片承載發光層之晶粒，做磁性巨量微整列與磁性巨量轉移，將加速Mini & Micr

▲蘋果指出，更新iOS 15後即使將手機重置，仍可使用尋找iPhone功能。（圖／取自9to5mac）
記者吳珍儀／綜合報導
蘋果稍早在WWDC 2021上宣布iOS 15將在今年秋季正式發布，蘋果表示，更新iOS 15後即使iPhone處於關機狀態，甚至是將手機重置，格式化至出廠設置，仍可使用尋找iPhone功能找出手機。
蘋果指出，若手機沒電關機，在幾個小時內仍能維持低功率消耗狀態，像AirTag原理一樣，透過藍牙訊號找回手機。
不過蘋果也指出，若想將尋找iPhone功能完全關閉，可在設置中關閉「低功耗模式」來關閉尋找iPhone功能。
----------------------------------------------------------------------

[5] 5月出口創歷來單月新高　財政部：4因素助攻

▲財政部今（8）日公布5月份出口創下歷年單月新高。（圖／記者吳佳穎攝）’
記者吳佳穎／台北報導
財政部今（8）日發布5 月海關進出口貿易初步統計， 5 月出口達 374.1 億美元，創歷年單月新高，較上月增 7.0%，年增 38.6%(按新臺幣計算增 29.7%)，是連續11個月正成長，也是11年以來的第二大增幅；累計 1 至 5 月出口 1,703.2 億美元，較上年同期增 30.2%(按新臺幣計算增 22.6%)。未來，如果疫情不致影響出口，第二季仍預估調升，出口維持擴張格局。
5 月進口為 312.5 億美元，為歷年單月次高，較上月增 8.6%，年增 40.9%(按新臺幣計算增 31.8%)；累計 1 至 5 月進口 1,436.1 億美元，較上年同期增 26.0%(按新臺幣計算增 18.6%)。
出入超部份， 5 月出超 61.6 億美元，較去年同月增 13.4 億美元；累計 1 至 5 月出 超 267.0 億美元，較去年同期增 98.6 億美元。
財政部統計處處長蔡美娜表示，5月份的出口是強力擴張格局，歸功四大因素：一、全球景氣逐步穩固復甦；二、國際原油、基本金屬原物料的行情帶動走高；三、新興科技應用、和居家與遠距商機亦持續發酵；四、出口產品的漲價效應。讓5 月出口規模再創新高，較去年同月增 104.1 億美元(+38.6%)。
蔡美娜指出，5月電子零組件出口 132.7 億美元，較去年同月增 30.3 億美元(+29.6

▲示意圖，與本文無關。（圖／取自免費圖庫Pexels）
網搜小組／周亭瑋報導
一名網友目前月薪8萬2，不過經常性加班，及工作可見度低、難升職等原因，讓他萌生離職的念頭；然而，不少人看完激動直呼，「你不想的話，請讓我進去」、「我願意去，歡迎跟我聯絡」。
網友在Dcard提到，他現職月薪8萬2，1年有14個月，外加股票年收約有130-140萬元，「可是很累！忙的時候，工時接近16小時，當然不會是每天，但在趕案子的時候，平均一天加班6小時，而且很難報加班費，因為底薪很高。」
他透露，有6大原因讓他想離職，除了工時很長、不能報加班外，也沒什麼能學的，「每天都在那邊拉Excel表格。」重點是，這份工作可見度低，高層完全看不到你的努力。
原po也忍不住抱怨，台灣分公司是很新的team，長官們也力求表現，而年資偏淺的他們，是很難有升職機會的，畢竟前面還有許多前輩在排隊，「我的直屬老闆是個面面俱到的人，每一邊他都不想得罪，所以更上層老闆開出的各種要求，他幾乎都接，接了就是我們這些下面的人在做。」
▲由於最近工作上遇到不少煩心事，讓他有了辭職的打算。（圖／翻攝自Dcard）
當然，高薪和好相處的同事，讓原po陷入兩難；另外，他也會擔心，「會不會以我的工作能力來說，能找到這樣的工作，就是封頂了？應該要繼續做下去比較好？」
文章引起熱烈迴響，網友們紛紛直呼，「你要不要辭職給我？我會用VBA，寫全自動更新的Excel報表」、「8萬2只要拉拉Excel？這誰都可以做吧」、「可以考CP王國營職員阿！大概20年就可以超越你現在的薪水了」、「請待在舒適圈爽好嗎？跳出來會發現自己犯傻啊」。
但也有人能理解，表示「感覺生活品質很差，我會選少一點錢，生活品質高一點的」、「我年薪也140左右，也是拉拉報表，工作好無聊，我也想離職，原來不只我這麼想」；當然，一些網友則質疑，是幻想文。
----------------------------------------------------------------------

[18] 疫情升溫、醫護人力不足　1111力挺推「醫院可免費刊登職缺3個月」

▲本土疫情升溫，雙北的醫護人員人力明顯不足。（圖／記者李毓康攝）
記者余弦妙／台北報導
新冠肺炎疫情不見趨緩，行政院長蘇貞昌宣布延長全國三級警戒，為期兩週至6月28日，根據人力銀行的調查發現，目前僅有4成上

▲京鼎原本預計停工兩日，今日將進行全員快篩，但因為快篩作業順延一日，停工時間也延長一日。（圖／取自京鼎官網）
記者高兆麟／綜合報導
昨日半導體設備廠商京鼎，竹南廠接獲六名外籍員工確診，公司表示已匡列相關接觸員工，並宣布停工兩天，不過原本預計今天就要進行全員快篩，但京鼎今日也發重訊表示，順延一天至9號才開始，停工也將增加一天，一共停工三日。
京鼎表示，公司原訂於6/8(二)進行企業快篩計畫，為完備快篩作業環節及保障員工及工作場所安全，將快篩作業順延一天至6/9(三)，停工將增加一天，企業預估停工三天。
----------------------------------------------------------------------

[30] 高端疫苗最快6／10解盲？　公司急發回應

▲高端澄清解盲目前與臨床試驗中心皆在盲性狀態下，交由第三方單位（CRO，臨床試驗受託機構）進行資料清理，預期近期內完成資料彙整，後續研發進度，公司將依規定進行公告。（圖／記者湯興漢攝）
記者吳康瑋／綜合報導
高端疫苗解盲前夕紛爭不斷，除了中研院院士陳培哲因擔心難以秉持獨立性及專業性，而請辭審查委員以外，還有媒體報導生技醫療產業人士認為，疫苗不太可能這麼快解盲，對此，高端於今（8）日中午緊急發布重訊，澄清解盲目前與臨床試驗中心皆在盲性狀態下，交由第三方單位（CRO，臨床試驗受託機構）進行資料清理，預期近期內完成資料彙整，並召開IDMC獨立資料監視委員會，以進行期間分析解盲作業，後續研發進度，公司將依規定進行公告。
高端指出，疫苗二期臨床主試驗，在4月28日完成所有受試者第二劑疫苗施打後，按藥證主管機關之技術性要求及臨床試驗設計，已於5月28日達到「所有受試者第二劑疫苗施打後一個月、以及中位數受試者追蹤二個月」之期間分析(Interim Analysis)條件。
----------------------------------------------------------------------

[31] 新MacBook Pro沒亮相！供應鏈證實：零組件短缺

▲新MacBook Pro沒在WWDC 2021中亮相。（圖／取自MacRumors）
記者廖婕妤／綜合報導
蘋果開發者大會WWDC 2021於台灣時間6月8日凌晨1時舉行，先前傳出搭載M1晶片的14吋及16吋M

▲高端先前召開二期臨床試驗收案完成說明會。（資料照／記者洪巧藍攝）
記者廖婕妤／綜合報導
高端疫苗（6547）二期臨床實驗結果最快預計在6月10日解盲，由於國內疫情升溫，造成疫苗需求量大幅增加，高端股價卻開始走下坡，從原先的最高點417元一路下跌，慘摔6根跌停，使股價直接腰斬；但今日跌停打開後，盤中卻又衝上228元漲停板。
法人表示，新藥研發時間長，能看到對公司營運實質貢獻的不確定性很高，但也因新藥股擁有的題材具想像空間，挾著本夢比吸引市場資金簇擁。
▲高端疫苗股價最高點曾來到417元。（圖／翻攝自三竹股市）
不過，從2014年的基亞（3176）到2016年的浩鼎（4174），新藥解盲前股價都創下新高價，但解盲結果不如預期，股價卻出現無量崩跌，嚇壞不少投資人，「炒作」質疑聲音不斷，也讓證交所因而要求相關公司必須增加透明度。
高端背後的大股東基亞（3176），在2014年7月時因為肝癌新藥「PI-88」第三期人體臨床實驗結果不如預期，股價從最高點486元跳空跌停，連續20天慘吞跌停板，套牢許多投資人。
浩鼎（4174）在2016年公告抗乳癌治療性疫苗「OBI-822」解盲結果卻不如預期，使股價從755元高點，因為內線交易等政治因素，一路慘摔至只剩58元。
除了解盲失敗，個股股價回歸原點，新藥解盲成功的個股，股價也不一定都能持盈保泰，2020年合一（4743）糖尿病足慢性傷口潰瘍新藥「ON101」，雖然解盲成功，但因市場聯想未來銷售產值有驚人數字，股價從31.55元狂飆至476.5元高點後，還是必須回到基本面的實質檢驗，最終股價僅能「利多出盡」回跌。
如今高端面臨同樣的困境，隨著官方「畫押」下半年國產疫苗將問市，儼然提前放出利多消息，但從新藥開發的歷程來看，解盲似乎沒有「大概、應該」這些模糊空間，也讓高端股價隨著市場眾多分析消息劇烈波動，在連跌6根停板後，8日盤中開低打開跌停，卻又戲劇性拉上漲停。
----------------------------------------------------------------------

[44] 不斷更新／東森集團、金融業力挺防疫前線　企業捐款破7億！

▲東森集團、聯邦銀、台新金、國泰金等企業，力挺防疫前線，紛紛捐款支持。（示意圖／資料照）
記者陳依旻／台北報導
台灣這次新冠肺炎 (COVID-19）疫情急速升溫，

記者陳心怡／台北報導
台股4月份外資淨匯入超過新台幣1400億元，帶動加權指數再創17709新高點，雖然大盤在創高後拉回整理，下跌2550點，最深跌幅達14.4%，但6月初又重新站回17300點之上，突顯出外資看好台股的堅定力量。這波驚驚漲行情，讓上市才半年的「020020」元大台股領航N緊跟台股表現，從發行價5元，快速漲到7.36元，又跌至6元後，再漲回7.22元，第二次逼近當初設定得提前贖回之停利點7.5元。
投資人擔心發行券商會執行停利機制不能繼續持有，紛紛打電話詢問。發行券商元大證券表示，元大台股領航N上市以來受到投資人熱烈支持，成長快速流通狀況極佳，為服務投資人可以長期投資台股，將不會執行提前贖回的停利機制。
元大台股領航N (020020)於去年11月掛牌，有單價低、專注投資台股龍頭的特性，目前只要新台幣七千多元，就能一次投資聯發科、台積電、台達電、國泰金、鴻海、廣達、富邦金、長榮、中華電、台塑化、台塑、南亞、大立光、華碩、台灣大等15檔龍頭股，享有和成分股指數相同的投資報酬率。
元大台股領航N之指標價值完全連結「特選台股領航報酬指數」表現。指數從台股核心產業中，選取各產業市值中最大的1到2支股票為成分股。2021年4月份成分股重新定審，新增華碩、長榮，刪除統一、中租KY。
▲元大台股領航N（020020）15檔指數成分股清單。（圖／元大提供）
「020020」包裹式的投資，具有低資金、免選股特性，可追蹤一籃子股票漲跌。核心價值在於能貼近「追蹤指數」的報酬。可以解決一般投資人資金較少，無法同時布局高價熱門股及無時間看盤選股的問題，且透過主題指數優勢的選股策略，不僅可以將有限資金專注在趨勢的強勢股上，還能分散風險。
----------------------------------------------------------------------

[53] 2030／50年零碳排放　台積電、玉山金控、信義房屋等企業共同倡議「淨零排放聯盟」

▲面對暖化挑戰，信義房屋與世界同步，自主揭露企業碳排放情形。
房產中心／綜合報導
全台一心，齊力抗疫，信義房屋不因疫情嚴峻而中斷減碳腳步，近日與台積電、玉山金控等全台13家永續標竿企業，一同宣示加入由財團法人台灣永續能源研究基金會所號召組成的「淨零排放聯盟」，承諾2030年達到信義房屋100%淨零排放(Net

▲高端結束6天跌停。（示意圖／記者湯興漢攝）
記者廖婕妤／綜合報導
高端疫苗（6547）二期尚未解盲，連收6根跌停，與最高峰417元相比，股價直接腰斬。今日開盤打開跌停，結束連6天跌停窘境，不過早盤股價表現仍相當弱勢，一度殺到190.5元，跌17元或8.19％，市值蒸發逾400億元。
台康生技（6589）止跌回穩，昨日盤中急拉漲停，終場收160元，上漲近10％；今日早盤翻黑，以157.5元開出，跌2.5元或1.5％。
----------------------------------------------------------------------

[66] 高端終結六天跌停！台股小漲10點　長榮、陽明漲幅逾3%

記者吳珍儀／綜合報導
美股今曇花一現，道瓊指數下跌逾百點，標準普爾指數則在盤中接近歷史高點後回檔。台股今（8）日以17094點開出，上漲10點，稍後漲勢擴大。
蘋概三王中，台積電（2330）今日以590元開出，下跌2元；大立光（3008）以3050元開出，下跌30元；鴻海（2317）則以111元開出，上漲0.5元。
今海運股、航空股強勢，長榮、陽明漲幅逾3%；長榮航漲幅逾5%、華航漲幅逾3%。
京元電（2449）股價今開盤下跌逾2%；超豐（2441）則上漲逾1%。此外，高端疫苗（6547）上週吞下第六根跌停後，今跌停終於打開，是高端疫苗連6個交易日跌停首次打開。
美股道瓊工業指數下跌126.15點，或0.36%，收34630.24點；標普 500 指數下跌3.37點，或0.08%，收4226.52點；那斯達綜合指數上漲67.23點，或0.49%，收13881.72點；費城半導體指數下跌22.49點，或0.70%，收3191.66點。
----------------------------------------------------------------------

[67] 影／機器人會做家事　還會幫你賺錢？

新聞節目中心／綜合報導
平時大家上班很忙碌，沒有時間看盤，常常收到對帳單才知道是賺是賠，更可能就這樣錯過了最美好的投資時機。有這樣困擾的投資人，該怎麼辦呢？富蘭克林投顧副總經理羅尤美建議，想要掌握時機，投資人需要一位專屬的理財顧問，隨時幫忙盯著市場，並且提醒何時該加碼、轉換或贖回基金，不管市場是漲或跌，都能提供投資人最適時的投

▲南化水庫。（圖／資料照）
記者林淑慧／台北報導
中央氣象局今（7）日針對台北市等九縣市發出大雨特報，挹注全台水庫豐沛雨量，依據經濟部水利署最新統計，這波梅雨鋒面的總降雨效益突破2.2億噸，讓水情獲得紓緩，自即日起，南投、高雄恢復水情正常，南彰化、雲林也轉為水情提醒綠燈。
官員指出，周末這波梅雨滯留鋒面所累積的大量降雨，約與上波鋒面降雨相當，而且最大降水量正好分布在最缺水的中南部，可謂下對了地方；依據統計，自4日迄今，南化水庫累計降雨量達307.8毫米，在全台水庫中排名冠軍。
蘭潭與仁義潭的進帳也不少，期間累計降雨量達293.5毫米，排名第二，贏過曾文與烏山頭水庫的279.5毫米、湖山水庫的250毫米、翡翠水庫的207.8毫米，推升南部地區成為這波降雨進帳最豐的區域。
如果以總降雨效益來觀察，南部的曾文與烏山頭水庫，截至今日晚間為止，預估將有6200萬噸雨量入庫，在全台主要水庫中排名第一；其次是北部的翡翠水庫預估將有4410萬噸雨量，而今天持續有強降雨的石門水庫，以1820萬噸水排名第三。
受惠兩波梅雨鋒面天降甘霖，至於全台水情何時解除旱災危機？經濟部表示，近期的及時雨對於長達一年的旱象有所緩解，但對整體水情的運用仍須精進，抗旱2.0計劃持續推進不能放鬆，除了高雄、南投已恢復水情正常外，其餘地區仍須節約用水；據估計，後續如果再有一場累計雨量超過百毫米的降雨，預計部分水情黃燈地區可望恢復正常供水。
----------------------------------------------------------------------

[80] 財政部、內政部：「房地合一2.0」7／1如期施行

中央社
商總主席賴正鎰呼籲，考量疫情對產業衝擊，房地合一稅2.0及實價登錄2.0應延期1年上路，對此，內政部與財政部均認為，延後必要性不大，目前無延期規劃，將如期於7月1 日上路。
兼任全國商業總會主席與鄉林董事長的賴正鎰今天發布新聞稿指出，政府打炒房政策發酵加上本土疫情升溫，建築業已成疫情「慘」業，建議即將於7月登場的房地合一稅2.0與實價登錄2.0，應延後1年實施。
花敬群說，多數業者對實價登錄2.0將在7月上路，「沒有覺得應該要延後」，內政部原則上會照原訂步驟推動。新政策上路初期，如果業者有登錄相關的問題，有些瑕疵也不會太苛刻，會給大家緩衝。
財政部高層則表示，首先

▲智邦科技公佈內部自結5月營收49.11億元，並指出全體外籍移工篩檢已結束，營運及生產出貨部份，皆維持正常運作。（圖／ETtoday新聞雲資料照）
記者吳康瑋／綜合報導
網路交換器大廠智邦（2345），於今（7）日公佈內部自結5月營收，49.11億元，較去年同期增加約30.49%，月增6.2%，截至110年 5月底止，累計營收為新台幣227.34億元，較去年同期增加約16.23%。
▲科技廠群聚 苗栗確診大爆發（圖／曾梓倩）
----------------------------------------------------------------------

[92] 影／65.5%民眾收入受疫情影響！東森保代挺你「自己當老闆」

新聞節目中心／綜合報導
疫情嚴峻，根據《ETtoday新聞雲》調查，65.5%民眾的收入受到疫情影響，政府更啟動紓困4.0挽救經濟。東森保代為了拚經濟，提高保險從業人員收入，即日起推出「自己當老闆」專案，只要加入該平台，每月支付3,000起的行政服務費用，就可自己當老闆，不僅領取100%佣金（需扣掉營業稅），同時還享有東森集團品牌資源，推展屬於自己的保險事業。
邁入後疫情時代，保險業務員該如何提振業績？東森保代祭出「自己當老闆」專案，提供保險業務員創業與就業機會，除了擁有一般保經代的保險業務資源，更享有東森集團的品牌資源，讓業務員不單是「一人作戰」，更是「軍團級戰略思維」。 因為一般小型保經公司可能知名度不足，欠缺業務推廣的品牌力道，若是加入東森保代，則可以運用東森集團「東森嚴選」的品牌力及媒體優勢。
東森保代顧問李育豪指出，自己開辦一家保經代，需要面對簽署人、辦公室租金、管理費、聘用行政助理、建置資訊系統等等費用，前期投入一、兩百萬跑不掉，且耗時一兩年，不確定因素高，且不一定會成功。但加入東森保代「自己當老闆」專案，創業成本極低，只要努力招攬業務，同時做好售後服務，加上東森集團的品牌優勢，不成功都很難，也不會面臨自己開公司倒閉的風險。
李育豪強調，東森保代「自己當老闆」是極佳的創業平台，對保險從業人員來說，是後疫情時代，值得發揮的新商模。
東森保代總經理李致琳說，東森保代願意讓利給業務人員，讓他們可以領到較高的收入，能夠藉此提高定著率，讓業務人員不會因為收入不穩定而跳槽新單位，此舉將有助於客戶權益之維護，除了讓客戶保單不會淪為孤

In [25]:
browser.close()